## Preprocessing of data about household head age for different household sizes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim 
from torchvision import datasets, transforms

In [2]:
data = pd.read_csv('C:/Users/Вера/Downloads/UNdata_Export_20240829_083912496.csv')

In [3]:
list_names = []
for i in range(len(data)):
    if (data['Age of head of household or other reference member'][i] != 'Total'
        and data['Sex of reference person'][i] == 'Total'
        and data['Household size'][i] == '1 person'
        and data['Country or Area'][i] not in list_names):
            list_names.append(data['Country or Area'][i])

In [4]:
arr = [j for j in range(len(data)) if sum([data['Country or Area'][j] == list_names[i] for i in range(len(list_names))])]

In [5]:
new_data = data.loc[arr]

In [6]:
dict_years = {name: [] for name in list_names}
for index in new_data.index:
    if new_data['Year'][index] not in dict_years[new_data['Country or Area'][index]]:
        dict_years[new_data['Country or Area'][index]].append(new_data['Year'][index])

In [7]:
for name in list_names:
    if len(dict_years[name]) > 1:
        for year in dict_years[name]:
            if len(new_data.loc[(new_data['Year'] == year) & (new_data['Country or Area'] == name)]) == len(new_data.loc[(new_data['Year'] == year) & (new_data['Country or Area'] == name) & (new_data['Age of head of household or other reference member'] == 'Total')]):
                indices = new_data.loc[(new_data['Year'] == year) & (new_data['Country or Area'] == name)].index
                new_data = new_data.drop(index=indices)

In [8]:
dict_names = {'10 - 14': [], '0 - 14': [], '12 - 14': [], '0 - 19': [], '15 - 19': [], '0 - 24': [], '0 - 15': [], '0 - 17': [], '16 - 19': [], '15 - 24': [], '20 - 24': [], '18 - 24': [], '25 - 29': [], '25 - 34': [], '30 - 34': [], '35 - 39': [], '40 - 44': [], '45 - 49': [], '50 - 54': [], '55 - 64': [], '55 - 59': [], '60 - 64': [], '65 - 69': [], '70 - 74': [], '65 - 74': [], '75+': [], '75 - 79': [], '65+': [], '80+': [], '80 - 84': [], '85 - 89': [], '90 - 94': [], '95 - 99': [], '85+': [], '100+': [], '95+': [], '90+': []}
for bracket in dict_names.keys():
    for index in new_data.loc[data['Age of head of household or other reference member'] == bracket].index:
        if data['Country or Area'][index] not in dict_names[bracket] and data['Country or Area'][index]:
            dict_names[bracket].append(data['Country or Area'][index])

In [9]:
dict_distr = {name: {'18 - 19': [[0 for i in range(6)], False], '20 - 24': [[0 for i in range(6)], False], '25 - 29': [[0 for i in range(6)], False], '30 - 34': [[0 for i in range(6)], False], '35 - 39': [[0 for i in range(6)], False], '40 - 44': [[0 for i in range(6)], False], '45 - 49': [[0 for i in range(6)], False], '50 - 54': [[0 for i in range(6)], False], '55 - 64': [[0 for i in range(6)], False], '65 - 74': [[0 for i in range(6)], False], '75 - 99': [[0 for i in range(6)], False]} for name in list_names}

In [10]:
for name in dict_names['15 - 19']:
    if name in dict_names['0 - 14']:
        if name not in ['Azerbaijan', 'Belarus', 'Georgia']:
            if not dict_distr[name]['18 - 19'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    dict_distr[name]['18 - 19'][0][number - 1] = summand1 + summand2
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['18 - 19'][0][0] = summand1 + summand2
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['18 - 19'][0][5] = summand1 + summand2
                dict_distr[name]['18 - 19'][1] = True
    elif name in dict_names['10 - 14']:
        if not dict_distr[name]['18 - 19'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '10 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['18 - 19'][0][number - 1] = summand1 + summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '10 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['18 - 19'][0][0] = summand1 + summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '10 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['18 - 19'][0][5] = summand1 + summand2
            dict_distr[name]['18 - 19'][1] = True
    elif name in dict_names['12 - 14']:
        if name not in ['Guinea-Bissau']:
            if not dict_distr[name]['18 - 19'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '12 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    dict_distr[name]['18 - 19'][0][number - 1] = summand1 + summand2
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '12 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['18 - 19'][0][0] = summand1 + summand2
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '12 - 14'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['18 - 19'][0][5] = summand1 + summand2
                dict_distr[name]['18 - 19'][1] = True
    else:
        if not dict_distr[name]['18 - 19'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                dict_distr[name]['18 - 19'][0][number - 1] = summand1
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            dict_distr[name]['18 - 19'][0][0] = summand1
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '15 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            dict_distr[name]['18 - 19'][0][5] = summand1
            dict_distr[name]['18 - 19'][1] = True

In [11]:
name = 'United Kingdom of Great Britain and Northern Ireland'
if not dict_distr[name]['18 - 19'][1]:
    for number in [2, 3, 4, 5]:
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 15'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand1 = df[0]
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '16 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['18 - 19'][0][number - 1] = summand1 + summand2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 15'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand1 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '16 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['18 - 19'][0][0] = summand1 + summand2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 15'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand1 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '16 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['18 - 19'][0][5] = summand1 + summand2
    dict_distr[name]['18 - 19'][1] = True

In [12]:
name = 'China, Macao SAR'
if not dict_distr[name]['18 - 19'][1]:
    for number in [2, 3, 4, 5]:
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '16 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['18 - 19'][0][number - 1] = summand2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '16 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['18 - 19'][0][0] = summand2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '16 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['18 - 19'][0][5] = summand2
    dict_distr[name]['18 - 19'][1] = True

In [13]:
for name in dict_names['0 - 19']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['18 - 19'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['18 - 19'][0][number - 1] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['18 - 19'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 19'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['18 - 19'][0][5] = summand2
            dict_distr[name]['18 - 19'][1] = True

In [14]:
for name in dict_names['0 - 24']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        for number in [2, 3, 4, 5]:
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['18 - 19'][0][number - 1] = summand2 // 2
            dict_distr[name]['20 - 24'][0][number - 1] = summand2 // 2
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['18 - 19'][0][0] = summand2 // 2
        dict_distr[name]['20 - 24'][0][0] = summand2 // 2
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['18 - 19'][0][5] = summand2 // 2
        dict_distr[name]['20 - 24'][0][5] = summand2 // 2
        dict_distr[name]['18 - 19'][1] = True
        dict_distr[name]['20 - 24'][1] = True

In [15]:
name = 'Suriname'
if not dict_distr[name]['18 - 19'][1]:
    for number in [2, 3, 4, 5]:
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 17'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand1 = df[0]
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '18 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['18 - 19'][0][number - 1] = summand1 + summand2 // 2
        dict_distr[name]['20 - 24'][0][number - 1] = summand2 // 2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 17'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '18 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['18 - 19'][0][0] = summand1 + summand2 // 2
    dict_distr[name]['20 - 24'][0][0] = summand2 // 2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '0 - 17'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '18 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['18 - 19'][0][5] = summand1 + summand2 // 2
    dict_distr[name]['20 - 24'][0][5] = summand2 // 2
    dict_distr[name]['18 - 19'][1] = True
    dict_distr[name]['20 - 24'][1] = True

In [16]:
for name in list_names:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['20 - 24'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '20 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['20 - 24'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '20 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['20 - 24'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '20 - 24'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['20 - 24'][0][5] = summand2
            dict_distr[name]['20 - 24'][1] = True

In [17]:
for name in dict_names['25 - 29']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['25 - 29'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '25 - 29'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['25 - 29'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '25 - 29'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['25 - 29'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '25 - 29'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['25 - 29'][0][5] = summand2
            dict_distr[name]['25 - 29'][1] = True

In [18]:
for name in list_names:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['30 - 34'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '30 - 34'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['30 - 34'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '30 - 34'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['30 - 34'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '30 - 34'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['30 - 34'][0][5] = summand2
            dict_distr[name]['30 - 34'][1] = True

In [19]:
for name in list_names:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['35 - 39'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '35 - 39'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['35 - 39'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '35 - 39'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['35 - 39'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '35 - 39'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['35 - 39'][0][5] = summand2
            dict_distr[name]['35 - 39'][1] = True

In [20]:
for name in list_names:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['40 - 44'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '40 - 44'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['40 - 44'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '40 - 44'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['40 - 44'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '40 - 44'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['40 - 44'][0][5] = summand2
            dict_distr[name]['40 - 44'][1] = True

In [21]:
for name in list_names:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['45 - 49'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '45 - 49'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['45 - 49'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '45 - 49'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['45 - 49'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '45 - 49'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['45 - 49'][0][5] = summand2
            dict_distr[name]['45 - 49'][1] = True

In [22]:
for name in list_names:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['50 - 54'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '50 - 54'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['50 - 54'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '50 - 54'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['50 - 54'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '50 - 54'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['50 - 54'][0][5] = summand2
            dict_distr[name]['50 - 54'][1] = True

In [23]:
for name in dict_names['55 - 59']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['55 - 64'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '55 - 59'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand1= df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '60 - 64'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['55 - 64'][0][number - 1] = summand1 + summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '55 - 59'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '60 - 64'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['55 - 64'][0][0] = summand1 + summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '55 - 59'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '60 - 64'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['55 - 64'][0][5] = summand1 + summand2
            dict_distr[name]['55 - 64'][1] = True

In [24]:
for name in [name for name in dict_names['65 - 69'] if name != 'Czechia']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['65 - 74'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 69'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand1= df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '70 - 74'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['65 - 74'][0][number - 1] = summand1 + summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 69'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '70 - 74'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['65 - 74'][0][0] = summand1 + summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 69'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '70 - 74'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['65 - 74'][0][5] = summand1 + summand2
            dict_distr[name]['65 - 74'][1] = True

In [25]:
for name in dict_names['65 - 74']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['65 - 74'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 74'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand = df[0]
                dict_distr[name]['65 - 74'][0][number - 1] = summand
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 74'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['65 - 74'][0][0] = summand2
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 74'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand2 = df[0]
            dict_distr[name]['65 - 74'][0][5] = summand2
            dict_distr[name]['65 - 74'][1] = True

In [26]:
for name in ['China, Hong Kong SAR', 'Oman', 'Suriname']:
    if not dict_distr[name]['65 - 74'][1]:
        for number in [2, 3, 4, 5]:
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
            df.index = [i for i in range(len(df))]
            summand = df[0]
            dict_distr[name]['65 - 74'][0][number - 1] = summand / 2
            dict_distr[name]['75 - 99'][0][number - 1] = summand / 2
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['65 - 74'][0][0] = summand2 / 2
        dict_distr[name]['75 - 99'][0][0] = summand2 / 2
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['65 - 74'][0][5] = summand2 / 2
        dict_distr[name]['75 - 99'][0][5] = summand2 / 2
        dict_distr[name]['65 - 74'][1] = True
        dict_distr[name]['75 - 99'][1] = True

In [27]:
name = 'Czechia'
if not dict_distr[name]['65 - 74'][1]:
    for number in [2, 3, 4, 5]:
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 69'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand1 = df[0]
        df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '70+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
        df.index = [i for i in range(len(df))]
        summand2 = df[0]
        dict_distr[name]['65 - 74'][0][number - 1] = summand1 + summand2 / 2
        dict_distr[name]['75 - 99'][0][number - 1] = summand2 / 2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 69'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand1 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '70+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['65 - 74'][0][0] = summand1 + summand2 / 2
    dict_distr[name]['75 - 99'][0][0] = summand2 / 2
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '65 - 69'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand1 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '70+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    dict_distr[name]['65 - 74'][0][5] = summand1 + summand2 / 2
    dict_distr[name]['75 - 99'][0][5] = summand2 / 2
    dict_distr[name]['65 - 74'][1] = True
    dict_distr[name]['75 - 99'][1] = True

In [28]:
for name in dict_names['75 - 79']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if name in dict_names['80 - 84'] and name in dict_names['85 - 89'] and name in dict_names['90 - 94'] and name in dict_names['95 - 99']:
            if not dict_distr[name]['75 - 99'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand3= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand4 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '95 - 99'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand5 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '100+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand6 = df[0]
                    dict_distr[name]['75 - 99'][0][number - 1] = summand1 + summand2 + summand3 + summand4 + summand5 + summand6
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand4 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '95 - 99'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand5 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '100+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand6 = df[0]
                dict_distr[name]['75 - 99'][0][0] = summand1 + summand2 + summand3 + summand4 + summand5 + summand6
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand4 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '95 - 99'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand5 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '100+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand6 = df[0]
                dict_distr[name]['75 - 99'][0][5] = summand1 + summand2 + summand3 + summand4 + summand5 + summand6
                dict_distr[name]['75 - 99'][1] = True

In [29]:
for name in dict_names['75 - 79']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if name in dict_names['80 - 84'] and name in dict_names['85 - 89'] and name in dict_names['90 - 94'] and name in dict_names['95+']:
            if not dict_distr[name]['75 - 99'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand3= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand4 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '95+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand5 = df[0]
                    dict_distr[name]['75 - 99'][0][number - 1] = summand1 + summand2 + summand3 + summand4 + summand5
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand4 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '95+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand5 = df[0]
                dict_distr[name]['75 - 99'][0][0] = summand1 + summand2 + summand3 + summand4 + summand5
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand4 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '95+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand5 = df[0]
                dict_distr[name]['75 - 99'][0][5] = summand1 + summand2 + summand3 + summand4 + summand5
                dict_distr[name]['75 - 99'][1] = True

In [30]:
for name in dict_names['75 - 79']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if name in dict_names['80 - 84'] and name in dict_names['85 - 89'] and name in dict_names['90+']:
            if not dict_distr[name]['75 - 99'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand3= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand4 = df[0]
                    dict_distr[name]['75 - 99'][0][number - 1] = summand1 + summand2 + summand3 + summand4
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand4 = df[0]
                dict_distr[name]['75 - 99'][0][0] = summand1 + summand2 + summand3 + summand4
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand4 = df[0]
                dict_distr[name]['75 - 99'][0][5] = summand1 + summand2 + summand3 + summand4
                dict_distr[name]['75 - 99'][1] = True

In [31]:
for name in dict_names['75 - 79']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if name in dict_names['80 - 84'] and name in dict_names['85+']:
            if not dict_distr[name]['75 - 99'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand3 = df[0]
                    dict_distr[name]['75 - 99'][0][number - 1] = summand1 + summand2 + summand3
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                dict_distr[name]['75 - 99'][0][0] = summand1 + summand2 + summand3
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand3 = df[0]
                dict_distr[name]['75 - 99'][0][5] = summand1 + summand2 + summand3
                dict_distr[name]['75 - 99'][1] = True

In [32]:
for name in dict_names['75 - 79']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if name in dict_names['80+']:
            if not dict_distr[name]['75 - 99'][1]:
                for number in [2, 3, 4, 5]:
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand1= df[0]
                    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                    df.index = [i for i in range(len(df))]
                    summand2 = df[0]
                    dict_distr[name]['75 - 99'][0][number - 1] = summand1 + summand2
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['75 - 99'][0][0] = summand1 + summand2
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
                df.index = [i for i in range(len(df))]
                summand2 = df[0]
                dict_distr[name]['75 - 99'][0][5] = summand1 + summand2
                dict_distr[name]['75 - 99'][1] = True

In [33]:
for name in dict_names['75+']:
    if name not in ['Azerbaijan', 'Belarus', 'Georgia', 'Guinea-Bissau', 'Puerto Rico', 'Sint Maarten (Dutch part)']:
        if not dict_distr[name]['75 - 99'][1]:
            for number in [2, 3, 4, 5]:
                df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
                df.index = [i for i in range(len(df))]
                summand1 = df[0]
                dict_distr[name]['75 - 99'][0][number - 1] = summand1
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            dict_distr[name]['75 - 99'][0][0] = summand1
            df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75+'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
            df.index = [i for i in range(len(df))]
            summand1 = df[0]
            dict_distr[name]['75 - 99'][0][5] = summand1
            dict_distr[name]['75 - 99'][1] = True

In [34]:
dict_distr['Kazakhstan']['75 - 99'][1] = True
dict_distr['Tokelau']['75 - 99'][1] = True

In [35]:
name = 'Latvia'
for number in [2, 3, 4, 5]:
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
    df.index = [i for i in range(len(df))]
    summand1= df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
    df.index = [i for i in range(len(df))]
    summand2 = df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
    df.index = [i for i in range(len(df))]
    summand3= df[0]
    df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == f'{number} persons']['Value']
    df.index = [i for i in range(len(df))]
    summand4 = df[0]
    dict_distr[name]['75 - 99'][0][number - 1] = summand1 + summand2 + summand3 + summand4
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
df.index = [i for i in range(len(df))]
summand1 = df[0]
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
df.index = [i for i in range(len(df))]
summand2 = df[0]
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
df.index = [i for i in range(len(df))]
summand3 = df[0]
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '1 person']['Value']
df.index = [i for i in range(len(df))]
summand4 = df[0]
dict_distr[name]['75 - 99'][0][0] = summand1 + summand2 + summand3 + summand4
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '75 - 79'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
df.index = [i for i in range(len(df))]
summand1 = df[0]
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '80 - 84'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
df.index = [i for i in range(len(df))]
summand2 = df[0]
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '85 - 89'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
df.index = [i for i in range(len(df))]
summand3 = df[0]
df = new_data.loc[new_data['Country or Area'] == name].loc[new_data['Age of head of household or other reference member'] == '90 - 94'].loc[new_data['Sex of reference person'] == 'Total'].loc[new_data['Household size'] == '6  persons and over']['Value']
df.index = [i for i in range(len(df))]
summand4 = df[0]
dict_distr[name]['75 - 99'][0][5] = summand1 + summand2 + summand3 + summand4
dict_distr[name]['75 - 99'][1] = True

In [39]:
arr_key = [key for key in dict_distr.keys()]

In [40]:
for key in arr_key:
    if not dict_distr[key]['18 - 19'][1]:
        del dict_distr[key]

In [45]:
dict_distr['Saint Barthelemy'] = dict_distr['Saint Barthélemy']
del dict_distr['Saint Barthélemy']

In [51]:
arr_key = [key for key in dict_distr.keys()]

In [50]:
with open('names_for_hh_by_age.txt', 'a') as file:
    for key in arr_key:
        file.write(key + '\n')

In [55]:
np.array([dict_distr['Russian Federation'][key][0] for key in dict_distr['Russian Federation'].keys()])

array([[ 478853.,  231660.,  177034.,  103846.,   27679.,   24991.],
       [ 554758.,  573657.,  544096.,  173878.,   47607.,   35835.],
       [ 489337.,  687846., 1218050.,  494426.,  104270.,   56092.],
       [ 462521.,  662987., 1317877.,  977558.,  204087.,   97531.],
       [ 480172.,  710768., 1345935., 1466001.,  347081.,  161877.],
       [ 661194., 1094947., 1780921., 1795971.,  489330.,  261198.],
       [ 766082., 1412915., 1732631., 1345738.,  462726.,  298115.],
       [ 913173., 1668880., 1428561.,  882309.,  380675.,  268526.],
       [2027027., 2965942., 1471223.,  832349.,  445396.,  322074.],
       [2855526., 3131118., 1019549.,  575556.,  355216.,  274489.],
       [2030495., 1383721.,  494143.,  291922.,  166679.,  120620.]])

In [56]:
for name in arr_key:
    np.savetxt(f'real samples of hh by age/{name}.txt', [dict_distr[name][key][0] for key in dict_distr[name].keys()])